<a href="https://colab.research.google.com/github/jacobamorgan/LWD_calcs/blob/main/Copy_of_LWD_jam_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Large Wood Stability Calculations

---
### Jacob A. Morgan, PhD, EIT
Reference: *Stream Habitat Restoration Guidlines (SHRG), Anchoring and Placement of Large Wood Appendix, 2012*

# Under the hood

Lasciate ogne speranza, voi ch'intrate

In [14]:
import numpy as np # for mathematical operations
import matplotlib.pyplot as plt # for plotting figures
from ipywidgets import widget, HTML, IntSlider, FloatSlider, FloatText, \
                       interact, Checkbox
from IPython.display import display, HTML
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))
################################################################################
def bldr_comp(D_bldr_in, N_bldr, Subm_bldr, gamma_boulder, gamma_water):
    # volume of boulder (ft^3)
    V_bldr_ft3 = (D_bldr_in/12)**3 # np.pi * (D_bldr/12)**3 / 6
    # dry weight of boulder (lbf)
    W_bldr_dry_lbf = gamma_boulder * V_bldr_ft3
    # wet weight of boulder (lbf)
    W_bldr_wet_lbf = (gamma_boulder-gamma_water) * V_bldr_ft3
    # weight of boulders (lbf)
    W_bldrs_lbf = N_bldr * Subm_bldr * W_bldr_wet_lbf + \
                  N_bldr * (1-Subm_bldr) * W_bldr_dry_lbf
    return V_bldr_ft3, W_bldr_dry_lbf, W_bldr_wet_lbf, W_bldrs_lbf
################################################################################
def log_vert(D_log_in, L_log_ft, gamma_wood, gamma_water, rootwad):
    # assume rootwad is 2 times log diameter
    if rootwad==True:
        L_rw_ft = 1.5 * (D_log_in / 12)
        D_rw_in = 2 * D_log_in
    else:
        L_rw_ft = 0.0
        D_rw_in = 0.0
    # area of log stem (ft^2)
    A_log_ft2 = np.pi * (D_log_in/12)**2 / 4
    # area of log rootwad (ft^2)
    A_rw_ft2 = np.pi * (D_rw_in/12)**2 / 4
    # volume of member (ft^3)
    V_log_ft3 = (L_log_ft-L_rw_ft) * A_log_ft2 + L_rw_ft * A_rw_ft2
    # weight of member (lbf)
    W_log_lbf = gamma_wood * V_log_ft3
    # buoyant force of member (lbf)
    F_B_log_lbf = V_log_ft3 * gamma_water - W_log_lbf
    return L_rw_ft, D_rw_in, A_rw_ft2, W_log_lbf, F_B_log_lbf
################################################################################
def log_horz(D_log_in, L_log_ft, L_rw_ft, D_rw_in, alpha, R_embed,\
             A_rw_ft2, v_1, grav, Subm_jam, C_DP, C_DU, mu_gravel, gamma_water,\
             D_soil_ft, W_bldrs_lbf, N_log, F_B_log_lbf, D_jam_ft, D_water_ft,\
             gamma_soil, gamma_soil_sat):
    # area of log normal to flow (ft^2)
    A_log_drag_ft2 = \
        np.sin(np.deg2rad(alpha))*(L_log_ft-L_rw_ft)*(1-R_embed)*D_log_in/12+\
        L_rw_ft*D_rw_in/12
    # drag force on log parallel to flow (lbf)
    F_D_log_lbf = (v_1**2)/2/grav*A_log_drag_ft2*Subm_jam*C_DP*gamma_water
    # drag force on rootwad normal to flow (lbf)
    F_D_rw_lbf = (v_1**2)/2/grav*A_rw_ft2*Subm_jam*C_DU*gamma_water
    # maximum drag force (lbf)
    F_D_single_lbf = max((F_D_log_lbf, F_D_rw_lbf)) * N_log
    # volume of soil over embedded log (ft^3)
    Vol_log_soil_ft3 = D_soil_ft*R_embed*L_log_ft*1.5*D_log_in/12
    # depth of dry soil (ft)
    D_soil_dry_ft = max((D_soil_ft+D_jam_ft-D_water_ft, 0.0))
    # depth of submerged soil (ft)
    D_soil_wet_ft = min((D_soil_ft, D_water_ft-D_jam_ft))
    # weight of soil over embedded log (lbf)
    W_log_soil_lbf = Vol_log_soil_ft3*(gamma_soil+D_soil_wet_ft/D_soil_ft*\
                                       (gamma_soil_sat-gamma_water-gamma_soil))
    # friction force (lbf)
    F_F_single_lbf = mu_gravel*(W_bldrs_lbf+N_log*(W_log_soil_lbf-F_B_log_lbf))
    return A_log_drag_ft2, W_log_soil_lbf, F_D_log_lbf, F_D_rw_lbf, \
        F_D_single_lbf, F_F_single_lbf
################################################################################
def FS_single(N_log, W_boulders_lbf, W_log_soil_lbf, F_B_log_lbf,\
              F_F_single_lbf, F_D_single_lbf):
    # uplift factor of safety
    FS_U_single = (W_boulders_lbf + W_log_soil_lbf*N_log)/N_log/F_B_log_lbf
    # drag factor of safety
    FS_D_single = F_F_single_lbf/F_D_single_lbf
    return FS_U_single, FS_D_single
################################################################################

def LWD_calcs(v_1, L_log_ft, D_log_in, N_log, rootwad, Subm_jam, alpha_deg,
              R_embed, D_jam_ft, W_jam_ft, L_jam_ft, D_bldr_in, N_bldr,
              Subm_bldr, D_water_ft, D_soil_ft):
    gamma_wood = 33 # Doug Fir dry unit weight (lbf/ft3)
    gamma_water = 62.4 # water unit weight (lbf/ft3)
    gamma_gravel_sat = 130 # streambed saturated unit weight (lbf/ft3)
    grav = 32.2 # gravitational acceleration (ft/s2)
    gamma_bldr = 165 # boulder unit weight (lbf/ft3)
    mu_gravel = 0.9 # coefficient of friction for gravel
    mu_sand = 0.4 # coefficient of friction for sand
    gamma_soil = 105 # dry soil density, assume gravel (lbf/ft3)
    gamma_soil_sat = 125 # saturated soil density, assume gravel (lbf/ft3)
    tau_pile = 330 # shear stress for pile (lbf/in2)
    C_DU = 1.2 # coefficient of drag (use 1.2 as suggested by WDFW)
    C_DP = 0.4
    phi_soil = 30 # angle of internal friction for non-cohesive soil (degrees)
    
    V_bldr_ft3, W_bldr_dry_lbf, W_bldr_wet_lbf, W_bldrs_lbf = \
        bldr_comp(D_bldr_in, N_bldr, Subm_bldr, gamma_bldr, gamma_water)
    L_rw_ft, D_rw_in, A_rw_ft2, W_log_lbf, F_B_log_lbf = \
        log_vert(D_log_in, L_log_ft, gamma_wood, gamma_water, rootwad)
    A_log_drag_ft2, W_log_soil_lbf, F_D_log_lbf, F_D_rw_lbf, \
        F_D_single_lbf, F_F_single_lbf = \
        log_horz(D_log_in, L_log_ft, L_rw_ft, D_rw_in, alpha_deg, R_embed, \
                 A_rw_ft2, v_1, grav, Subm_jam, C_DP, C_DU, mu_gravel, \
                 gamma_water, D_soil_ft, W_bldrs_lbf, N_log, F_B_log_lbf, \
                 D_jam_ft, D_water_ft, gamma_soil, gamma_soil_sat)
    FS_U_single, FS_D_single = \
        FS_single(N_log, W_bldrs_lbf, W_log_soil_lbf, F_B_log_lbf, \
                  F_F_single_lbf, F_D_single_lbf)
    
    alpha_deg_arr = np.matrix(np.arange(0,181,2))
    D_bldr_in_arr = np.matrix(np.arange(24,40.1,0.5)).T
    # R_embed_arr = np.matrix(np.arange(0,0.61,2)).T
    [xg, yg] = np.meshgrid(alpha_deg_arr, D_bldr_in_arr)
    FS_U_single_arr = np.single(np.zeros_like(xg))
    FS_D_single_arr = np.single(np.zeros_like(xg))
    for x in np.uintc(np.linspace(0,np.size(alpha_deg_arr)-1,np.size(alpha_deg_arr))):
        for y in np.uintc(np.linspace(0,np.size(D_bldr_in_arr)-1,np.size(D_bldr_in_arr))):
            alpha_deg_a = alpha_deg_arr[0,x]
            D_bldr_in_a = D_bldr_in_arr[y,0]

            V_bldr_ft3, W_bldr_dry_lbf, W_bldr_wet_lbf, W_bldrs_lbf = \
                bldr_comp(D_bldr_in_a, N_bldr, Subm_bldr, gamma_bldr, gamma_water)
            
            L_rw_ft, D_rw_in, A_rw_ft2, W_log_lbf, F_B_log_lbf = \
                log_vert(D_log_in, L_log_ft, gamma_wood, gamma_water, rootwad)
            
            A_log_drag_ft2, W_log_soil_lbf, F_D_log_lbf, F_D_rw_lbf, \
            F_D_single_lbf, F_F_single_lbf = \
                log_horz(D_log_in, L_log_ft, L_rw_ft, D_rw_in, alpha_deg_a, \
                        R_embed,A_rw_ft2, v_1, grav, Subm_jam, C_DP, C_DU, \
                        mu_gravel, gamma_water, D_soil_ft, W_bldrs_lbf, N_log, \
                        F_B_log_lbf, D_jam_ft, D_water_ft, gamma_soil, \
                        gamma_soil_sat)
            
            FS_U_single_arr[y,x], FS_D_single_arr[y,x] = \
                FS_single(N_log, W_bldrs_lbf, W_log_soil_lbf, F_B_log_lbf, \
                          F_F_single_lbf, F_D_single_lbf)

    plt.style.use('classic')
    plt.style.use('ggplot')
    plt.rcParams.update({'font.size': 14})
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
    for i, ax in enumerate(axes.flat):
        if i == 0:
            zdat = FS_U_single_arr
            ax.set_title(r'Uplift, $\it{FS_U}$ = '+str(round(FS_U_single,2)))
        elif i == 1:
            zdat = FS_D_single_arr
            ax.set_title(r'Drag, $\it{FS_D}$ = '+str(round(FS_D_single,2)))
        CS = ax.contourf(xg, yg, zdat,
                      levels = np.append(np.insert(np.arange(0,4.5,0.5),0,-0.25),4.25),
                      extend = 'both',
                      cmap = plt.cm.RdYlGn,
                      alpha = 0.3)
        if i == 0:
            cbar = fig.colorbar(CS, ax=axes[:].ravel().tolist())
            cbar.ax.set_ylabel(r'Factor of Safety')
        if np.min(zdat)<=2.0 and np.max(zdat)>=2.0:
            CSL = ax.contour(xg, yg, zdat,
                             levels = [2],
                             colors = 'k',
                             linestyles = '--',
                             linewidths = 4)
            cbar.add_lines(CSL)
            cbar.lines[-1].set_linestyles(CSL.linestyles)
        ax.set_xlabel(r'log orientation, $\it\alpha$ ($^\circ$)')
        ax.set_xticks(np.arange(0,365,30))
        ax.tick_params(axis='x', labelrotation=45)
        ax.grid(color='k', alpha=0.25)
        ax.plot(alpha_deg, D_bldr_in, 'Xk', markersize=20)
    
    plt.setp(axes[0,], ylabel=r'boulder diameter, $\it{D_{bldr}}$ (in)')
    plt.show()

    out = HTML('<p style="font-size:24px">FS<sub>U,single</sub> = '+str(round(FS_U_single,2))+'<p>'+\
               '<p style="font-size:24px">FS<sub>D,single</sub> = '+str(round(FS_D_single,2))+'<p>')
    display(out)

    # return FS_U_single, FS_D_single
################################################################################
style = {'description_width': 'initial'}
layout = {'width': '500px'}
inputs = interact(LWD_calcs,
         v_1 = FloatText(value=12.0, min=0, step=0.1,
                         description='Design velocity, v_1 (ft/s)',
                         style=style, layout=layout,
                         continuous_update=True),
         L_log_ft = IntSlider(value=20, min=5, max=35, step=5,
                              description='Length of log, L_log (ft)',
                              style=style, layout=layout,
                              continuous_update=True),
         D_log_in = IntSlider(value=18, min=8, max=36, step=2,
                              description='Diameter of log, D_log (in)',
                              style=style, layout=layout,
                              continuous_update=True),
         N_log = IntSlider(value=1, min=0, max=10, step=1,
                           description='Number of logs, N_log',
                           style=style, layout=layout,
                           continuous_update=True),
         rootwad = Checkbox(value=True,
                            description='Include rootwad?',
                            style=style, layout=layout,
                            continuous_update=True),
         Subm_jam = FloatSlider(value=1, min=0.0, max=1.0, step=0.1,
                                description='Fraction of jam submerged, Subm_jam',
                                style=style, layout=layout,
                                continuous_update=True),
         alpha_deg = IntSlider(value=90, min=0, max=180, step=15,
                               description='Orientation of log, alpha (degrees)',
                               style=style, layout=layout,
                               continuous_update=True),
         R_embed = FloatSlider(value=0, min=0.0, max=1.0, step=0.1,
                               description='Fraction of log embedded, R_embed',
                               style=style, layout=layout,
                               continuous_update=True),
         D_jam_ft = FloatText(value=3.0, min=0, max=20, step=0.1,
                              description='Depth of jam, D_jam (ft)',
                              style=style, layout=layout,
                              continuous_update=True),
         W_jam_ft = FloatText(value=3.0, min=0, max=50, step=0.1,
                              description='Width of jam, W_jam (ft)',
                              style=style, layout=layout,
                              continuous_update=True),
         L_jam_ft = FloatText(value=3.0, min=0, max=50, step=0.1,
                              description='Length of jam, L_jam (ft)',
                              style=style, layout=layout,
                              continuous_update=True),
         D_bldr_in = IntSlider(value=32, min=8, max=48, step=2,
                               description='Diameter of boulder, D_bldr (in)',
                               style=style, layout=layout,
                               continuous_update=True),
         N_bldr = IntSlider(value=2, min=0, max=4, step=1,
                            description='Number of boulders, N_bldr',
                            style=style, layout=layout,
                            continuous_update=True),
         Subm_bldr = FloatSlider(value=1, min=0.0, max=1.0, step=0.1,
                                 description='Fraction of boulder submerged, Subm_bldr',
                                 style=style, layout=layout,
                                 continuous_update=True),
         D_water_ft = FloatText(value=3.0, min=0, max=20, step=0.1,
                                description='Depth of water, D_water (ft)',
                                style=style, layout=layout,
                                continuous_update=True),
         D_soil_ft = FloatText(value=3.0, min=0, max=20, step=0.1,
                               description='Depth of soil, D_soil (ft)',
                               style=style, layout=layout,
                               continuous_update=True))




interactive(children=(FloatText(value=12.0, continuous_update=True, description='Design velocity, v_1 (ft/s)',…

The first function calculates the volume and weight for the boulder components.  
Boulder volume, assuming a cube shape, is calculated as: $V_{bldr} = D_{bldr}^3$  
Boulder dry weight: $W_{bldr,dry} = \gamma_{bldr}\cdot V_{bldr}$  
Boulder wet weight: $W_{bldr,wet} = (\gamma_{bldr}-\gamma_{water})\cdot V_{bldr}$  
The total weight of boulders is computed using: $W_{bldrs} = N_{bldr}\cdot Subm_{bldr}\cdot W_{bldr,wet}+N_{bldr}\cdot(1-Subm_{bldr})\cdot W_{bldr,dry}$

The next function computes the vertical forces acting on log members.

If the member has a rootwad, the rootwad length is assumed to 1.5$\times$ the diameter of the log, and the rootwad diameter is assumed to be 2$\times$ the diameter of the log. To compute the volume of the log, we first calculate the cross-section area of the log stem and the rootwad.  
Area of log stem: $A_{log} = \pi\cdot D_{log}^2/4$  
Area of rootwad: $A_{rootwad} = \pi\cdot D_{rootwad}^2/4$  
The total volume of the log can be determined using: $V_{log} = (L_{log}-L_{rootwad})\cdot A_{log} + L_{rootwad}\cdot A_{rootwad}$

The downward force, or weight, of the log is: $W_{log} = \gamma_{wood}\cdot V_{log}$

The upward, or buoyant, force of the log is: $F_{B,log} = V_{log}\cdot\gamma_{water}-W_{log}$

The next function computes the horizontal forces acting on log members.

Area of the log acted on the flow (normal to flow direction): $A_{log,drag} = \sin\alpha\cdot(L_{log}-L_{rootwad})\cdot(1-R_{embed})\cdot D_{log}+L_{rootwad}\cdot D_{rootwad}$  
Drag on the log parallel to the flow: $F_{D,log} = \displaystyle\frac{v_1^2}{2\cdot g}\cdot A_{log,drag}\cdot Subm_{jam}\cdot C_{DP}\cdot\gamma_{water}$  
Drag on the rootwad normal to the flow: $F_{D,rootwad} = \displaystyle\frac{v_1^2}{2\cdot g}\cdot A_{rootwad}\cdot Subm_{jam}\cdot C_{DU}\cdot\gamma_{water}$  
Maximum drag on the member: $F_{D,single} = \max\left(\left[\begin{matrix}F_{D,log} \\ F_{D,rootwad}\end{matrix}\right]\right)\cdot N_{log}$  
Volume of soil on the embedded log portion: $V_{log,soil} = D_{soil}\cdot R_{embed}\cdot L_{log}\cdot 1.5\cdot D_{log}$  
Depth of dry soil on log: $D_{soil,dry} = \max\left(\left[\begin{matrix} D_{soil}+D_{jam}-D_{water} \\ 0 \end{matrix}\right]\right)$  
Depth of submerged soil: $D_{soil,wet} = \min\left(\left[\begin{matrix} D_{soil} \\ D_{water}-D_{jam} \end{matrix}\right]\right)$  
Weight of soil on the embedded log: $W_{log,soil} = V_{log,soil}\cdot\left[\gamma_{soil}+\displaystyle\frac{D_{soil,wet}}{D_{soil}}\cdot\left(\gamma_{soil,sat}-\gamma_{water}-\gamma_{soil}\right)\right]$  
Friction force on members: $F_{F,single} = \mu_{gravel}\cdot\left[W_{bldrs}+N_{log}\cdot\left(W_{log,soil}-F_{B,log}\right)\right]$